# **Esame del 16 febbraio 2024**

- Non si possono consultare libri, note, ed ogni altro materiale o persone durante
l'esame ad eccezione delle funzioni Python fornite.

- Risolvere gli esercizi marcati con **[P]** all'interno dell'IPython notebook.

- Svolgere gli esercizi marcati con **[T]** su fogli protocollo, indicando: nome, cognome, codice persona e data

- La durata del compito è di 90 minuti.

- Questo esame è composto da 3 esercizi, per un totale di 30/30 punti.

- All'interno degli script **functions2024.py** e **fem.py** trovate le funzioni necessarie per gli esercizi (che potete quindi importare ed utilizzare liberamente).


In [ ]:
# Cella di inizializzazione. Eseguirla prima di iniziare lo svolgimento dell'esame (ed, eventualmente, ad ogni riavvio).
import matplotlib.pyplot as plt
import numpy as np
from fem import install

install()

## **Esercizio 1** &nbsp; &nbsp; <small>(10 punti)</small>

Si consideri la seguente matrice

$$A=\left[\begin{array}{ccccc}1&1&1&1&1\\ 1&2&3&4&5\\ 1& 3&6&10&15 \\ 1&4&10&20&35\\ 1&5&15&35&70\end{array}\right]$$

che si può costruire con il comando scipy.linalg.pascal(n), dove n è la dimensione della matrice.

##### **a) [P+T]** &nbsp; &nbsp; (3 punti)

Enunciare la condizione necessaria e sufficiente per l'esistenza e unicità della fattorizzazione LU e verificare (con opportuni comandi) che è soddisfatta per la matrice $A$ di dimensione $n = 5$.

In [ ]:
from scipy.linalg import pascal




##### **b) [P]** &nbsp; &nbsp; (3 punti)

 Data la soluzione esatta $\mathbf{x} = [1, 1 . . . , 1]^T$ costruire il termine noto $\mathbf{b}$
e risolvere il sistema lineare $A\mathbf{x} = \mathbf{b}$ utilizzando
la fattorizzazione LU seguita dai metodi di sostituzione in avanti e all'indietro (si sfruttino le funzioni fornite). Verificare se è stato effettuato il pivoting.

In [ ]:
from functions2024 import fwsub, bksub
from scipy.linalg import lu




##### **c) [P]** &nbsp; &nbsp; (4 punti)

Ripetere i passaggi al punto precedente per matrici di dimensione $n = 10$, $15$ e $20$. Per ognuno dei casi calcolare la norma dell'errore relativo e il condizionamento della matrice e rappresentarli su due grafici in scala logaritmica. Commentare i risultati alla luce della teoria.

## **Esercizio 2** &nbsp; &nbsp; <small>(10 punti)</small>

Si consideri il seguente integrale

$$I=\displaystyle \int_a^b f(x) dx$$

con $f(x)=\dfrac{1}{\sqrt{x^2-1}}$, sapendo che la primitiva dell'integranda è data da
$$F(x) = |x+\sqrt{x^2-1}|+c.$$

##### **a) [T]** &nbsp; &nbsp; (4 punti)

Introdurre il metodo di quadratura del punto medio e di Simpson e la loro versione composita. Qual è l'ordine di accuratezza per tali metodi? E il loro grado di esattezza?

##### **b) [P+T]** &nbsp; &nbsp; (4 punti)

Usando le funzioni pmedcomp e simpcomp calcolare un'approssimazione di $I$ considerando l'intervallo di integrazione compreso fra $a = 1.02$ e $b = 2$, e suddividendolo in $N = [10, 20, 40, 80]$ sotto-intervalli. Calcolare l'errore ottenuto e rappresentarlo in scala logaritmica. Commentare i risultati ottenuti alla luce della teoria.

In [ ]:
from functions2024 import pmedcomp, simpcomp





##### **c) [P+T]** &nbsp; &nbsp; (2 punti)

Rappresentare graficamente l'integranda $f(x)$ nell'intervallo indicato, e proporre una strategia per la scomposizione dell'intervallo di integrazione in modo da aumentare l'accuratezza mantenendo lo stesso numero di divisioni.

## **Esercizio 3** &nbsp; &nbsp; <small>(10 punti)</small>

Si consideri il seguente problema

\begin{gather*}
    \begin{cases}
        -u^{\prime\prime} = f(x) & 0< x< 1, \\
        u(0)=0                              \\
        u^\prime(1)= 4e^2
    \end{cases}
\end{gather*}

##### **a) [T]** &nbsp; &nbsp; (1 punto)

Determinare $f$ tale per cui $u(x) = e^{2x}x^2$ è soluzione esatta del problema e verificare che tale soluzione effettivamente rispetta le condizioni al bordo.

##### **b) [T]** &nbsp; &nbsp; (4 punti)

Introdurre brevemente l'approssimazione del problema con il metodo
degli elementi finiti in spazio e derivarne l'espressione matriciale considerando le condizioni al contorno del problema in esame.

##### **c) [P]** &nbsp; &nbsp; (2 punti)

Si consideri una griglia di ampiezza uniforme $h = 0.1$. Si risolva il
problema con il metodo degli elementi finiti lineari, utilizzando la function dirNeusolve fornita. Si rappresenti in un grafico la soluzione numerica sovrapposta alla soluzione esatta.

In [ ]:
from fem import Line, generate_mesh, FEspace, plot
from fem import dx, ds, deriv, assemble, interpolate, DirichletBC, applyBCs, dof2fun
from scipy.sparse.linalg import spsolve

# Dati del problema
domain = Line(0, 1)
f = lambda x:  -2*np.exp(2*x)*(2*x**2 + 4*x + 1)

def dirNeusolve(alpha, gamma, h):
  """
  Risolve il problema -u'' = f per 0 < x < 1 con condizioni miste Dirichlet-Neumann
  implementando il metodo agli elementi finiti (P1).

  Input:
    alpha    (float)     condizione di Dirichlet sul bordo sinistro, u(0) = alpha.
    gamma    (float)     condizione di Neumann sul bordo destro, u(1) = gamma.
    h        (float)     passo della mesh

  Output:
    u (function di FEniCS).
  """

  mesh = generate_mesh(domain, stepsize = h)
  V = FEspace(mesh, 1)

  fh = interpolate(f, V)
  phi = lambda x: gamma
  phih = interpolate(phi, V)

  def l(v):
    return fh*v*dx + phih*v*ds

  def a(u, v):
    return deriv(u)*deriv(v)*dx

  A = assemble(a, V)
  F = assemble(l, V)

  def isLeftNode(x):
    return x < 1e-12

  dbc = DirichletBC(isLeftNode, alpha)
  A = applyBCs(A, V, dbc)
  F = applyBCs(F, V, dbc)

  u = spsolve(A, F)
  u = dof2fun(u, V)
  return u

##### **d) [P]** &nbsp; &nbsp; (3 punti)

Si ripeta il calcolo della soluzione numerica al variare
di $h$ con $h =1/N$ , $N = 80, 160, 320$. Calcolare l'errore e rappresentarne l'andamento in scala logaritmica. Commentare il risultato alla luce della teoria.

In [ ]:
from fem import L2error



